In [1]:
# questions I have as I think about this more:
# when we do the whole deduplication, we should only be doing that on TRAIN, right?

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
import pandas as pd
import numpy as np
import copy
import time
import pickle

In [3]:
splits = "splits_1571429760//"

In [4]:
data_folder = "data//"
train_path = data_folder + splits + "train.csv"
dev_path = data_folder + splits + "dev.csv"

In [5]:
train_df = pd.read_csv(train_path, index_col=0)

/home/emilyrapport/anaconda3/envs/w266-env/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
dev_df = pd.read_csv(dev_path, index_col=0)

In [7]:
vectorizer = HashingVectorizer()

In [8]:
responses = train_df.response_text
xs = []
for i in range(1,11):
    print("starting chunk {}".format(i))
    chunk = responses.iloc[(i-1)*1000000:i*1000000]
    X = vectorizer.fit_transform(chunk.apply(lambda x: np.str_(x)))
    xs.append(X)

starting chunk 1
starting chunk 2
starting chunk 3
starting chunk 4
starting chunk 5
starting chunk 6
starting chunk 7
starting chunk 8
starting chunk 9
starting chunk 10


In [9]:
X = vstack(xs)

In [10]:
X.shape

(9879016, 1048576)

In [11]:
Y = train_df.op_gender.values

In [12]:
model = LogisticRegression(random_state=0)

In [15]:
time_start = time.time()


model.fit(X,Y)
currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Model trained in {}".format(timeStr))

/home/emilyrapport/anaconda3/envs/w266-env/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Model trained in 31 minutes, 08 seconds


In [16]:
dev_x = vectorizer.fit_transform(dev_df.response_text.apply(lambda x: np.str_(x)))

In [17]:
dev_y = dev_df.op_gender.values

In [19]:
## what??? that's too high
model.score(dev_x, dev_y)

0.824028187798284

In [26]:
pkl_filename = "models//first_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)